In [1]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt

import pythiademo
from pythiademo import PythiaDemo

import json
import pandas as pd
from data.tokenizer.ptbtokenizer import PTBTokenizer


%load_ext autoreload
%autoreload 2

---------- BASE_PATH '/tf/notebooks/jwhwang/vqa/mypythia'


In [2]:
# glob("./data/imdb/coco_captions/imdb_karpathy_train.npy")
imdb_train = np.load("./data/imdb/coco_captions/imdb_karpathy_train.npy", allow_pickle=True)
imdb_test = np.load("./data/imdb/coco_captions/imdb_karpathy_test.npy", allow_pickle=True)
imdb_val = np.load("./data/imdb/coco_captions/imdb_karpathy_val.npy", allow_pickle=True)

In [16]:
imdb_train[:2]

array([{'metadata': 'coco'},
       {'image_name': 'COCO_train2014_000000057870', 'image_id': 40504, 'feature_path': 'COCO_train2014_000000057870.npy', 'caption_id': 787980, 'caption_str': 'A restaurant has modern wooden tables and chairs.', 'caption_tokens': ['<s>', 'a', 'restaurant', 'has', 'modern', 'wooden', 'tables', 'and', 'chairs', '</s>'], 'reference_tokens': [['<s>', 'a', 'restaurant', 'has', 'modern', 'wooden', 'tables', 'and', 'chairs', '</s>'], ['<s>', 'a', 'long', 'restaurant', 'table', 'with', 'rattan', 'rounded', 'back', 'chairs', '</s>'], ['<s>', 'a', 'long', 'table', 'with', 'a', 'plant', 'on', 'top', 'of', 'it', 'surrounded', 'with', 'wooden', 'chairs', '</s>'], ['<s>', 'a', 'long', 'table', 'with', 'a', 'flower', 'arrangement', 'in', 'the', 'middle', 'for', 'meetings', '</s>'], ['<s>', 'a', 'table', 'is', 'adorned', 'with', 'wooden', 'chairs', 'with', 'blue', 'accents', '</s>']]}],
      dtype=object)

In [18]:
imdb_train[-2:]

array([{'image_name': 'COCO_train2014_000000475546', 'image_id': 123286, 'feature_path': 'COCO_train2014_000000475546.npy', 'caption_id': 8002, 'caption_str': 'Group of people drinking wine at a public location.', 'caption_tokens': ['<s>', 'group', 'of', 'people', 'drinking', 'wine', 'at', 'a', 'public', 'location', '</s>'], 'reference_tokens': [['<s>', 'the', 'patrons', 'enjoy', 'their', 'beverages', 'at', 'the', 'bar', '</s>'], ['<s>', 'people', 'having', 'a', 'drink', 'in', 'a', 'basement', 'bar', '</s>'], ['<s>', 'a', 'group', 'of', 'friends', 'enjoys', 'a', 'drink', 'while', 'sitting', 'at', 'a', 'bar', '</s>'], ['<s>', 'group', 'of', 'people', 'drinking', 'wine', 'at', 'a', 'public', 'location', '</s>'], ['<s>', 'three', 'women', 'and', 'a', 'man', 'are', 'sitting', 'at', 'a', 'bar', '</s>']]},
       {'image_name': 'COCO_train2014_000000475546', 'image_id': 123286, 'feature_path': 'COCO_train2014_000000475546.npy', 'caption_id': 9277, 'caption_str': 'Three women and a man are si

In [3]:
with open('./data/vizwiz/annotations/train.json', "r", encoding="utf-8") as f:
    train_json = json.load(f)
    
# with open('./data/vizwiz/annotations/test.json', "r") as f:
#     test_json = json.load(f)    
    
with open('./data/vizwiz/annotations/val.json', "r", encoding="utf-8") as f2:
    val_json = json.load(f2)    

In [4]:
# get images for train
img_anno = []
for img_str in train_json["images"]:
    img_anno.append({"image_id":img_str["id"], "image_name":img_str["file_name"].split(".")[0], "file_name":img_str["file_name"]})
    
# get annos for train
anno_anno = []
for ann in train_json["annotations"]:
    anno_anno.append({"image_id":ann["image_id"], "caption_id":ann["id"], "caption_str":ann["caption"]})
    
# get images of val
val_img_anno = []
for val_img_str in val_json["images"]:
    val_img_anno.append({"image_id":val_img_str["id"], "image_name":val_img_str["file_name"].split(".")[0], "file_name":val_img_str["file_name"]})

# get annos for val
val_anno_anno = []
for val_ann in val_json["annotations"]:
    val_anno_anno.append({"image_id":val_ann["image_id"], "caption_id":val_ann["id"], "caption_str":val_ann["caption"]})

In [5]:
# for train
img_anno_df = pd.DataFrame(img_anno)
anno_anno_df = pd.DataFrame(anno_anno)
anno_df = anno_anno_df.join(img_anno_df, on="image_id", rsuffix="i_")
anno_df = anno_df.drop("image_idi_", axis=1)

anno_df.head(1)

,image_id,caption_id,caption_str,image_name,file_name
0,0,0,ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS T...,VizWiz_train_00000000,VizWiz_train_00000000.jpg


In [6]:
# for val
val_img_anno_df = pd.DataFrame(val_img_anno)
val_anno_anno_df = pd.DataFrame(val_anno_anno)
# val_anno_df = val_anno_anno_df.join(val_img_anno_df, on="image_id", rsuffix="i_")
# val_anno_df = val_anno_df.drop("image_idi_", axis=1)
val_anno_df = pd.merge(val_img_anno_df, val_anno_anno_df)

In [8]:
val_anno_df.head(1)

,image_id,image_name,file_name,caption_id,caption_str
0,23431,VizWiz_val_00000000,VizWiz_val_00000000.jpg,117155,A computer screen shows a repair prompt on the...


In [9]:
def split_caption(caption:str):
    return ['<s>', *caption.split(" "), '</s>']

In [10]:
anno_df["caption"] = anno_df["caption_str"].map(split_caption)
val_anno_df["caption"] = val_anno_df["caption_str"].map(split_caption)
# anno_df = anno_df.drop("caption_str", axis=1)

In [11]:
group_idx = anno_df.groupby(anno_df["image_id"]).indices
val_group_idx = val_anno_df.groupby(val_anno_df["image_id"]).indices

def make_references_tokens(group_idx, image_id:int):
    return list(anno_df["caption"].iloc[group_idx[image_id]])

anno_df["reference_tokens"] = anno_df["image_id"].map(lambda x: make_references_tokens(group_idx, x))

In [12]:
val_anno_df["reference_tokens"] = val_anno_df["image_id"].map(lambda x: make_references_tokens(val_group_idx, x))

In [31]:
def make_annotation(img_name:str, img_id:int, file_name:str, caption_id:int, caption_str:str, caption:list, reference_tokens:list):
    return {'image_name':img_name, 'image_id':img_id, 'feature_path':file_name, 'caption_id':caption_id, 'caption_str':caption_str, 'caption_tokens':caption, 'reference_tokens':reference_tokens}

anno_df["result"] = anno_df.apply(lambda x: make_annotation(img_name=x["image_name"], img_id=x["image_id"],
                                       file_name=x["file_name"], caption_id=x["caption_id"], 
                                       caption_str=x["caption_str"], caption=x["caption"],
                                       reference_tokens=x["reference_tokens"]), axis=1)

val_anno_df["result"] = val_anno_df.apply(lambda x: make_annotation(img_name=x["image_name"], img_id=x["image_id"],
                                       file_name=x["file_name"], caption_id=x["caption_id"], 
                                       caption_str=x["caption_str"], caption=x["caption"],
                                       reference_tokens=x["reference_tokens"]), axis=1)

In [32]:
# vizwiz_anno = [{'metadata': 'vizwiz'}, *anno_df["result"]]
# val_vizwiz_anno = [{'metadata': 'vizwiz'}, *anno_df["result"].iloc[:-100]]
# test_vizwiz_anno = [{'metadata': 'vizwiz'}, *anno_df["result"].iloc[-100:]]

In [24]:
anno_df.head(1)

,image_id,caption_id,caption_str,image_name,file_name,caption,reference_tokens
0,0,0,ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS T...,VizWiz_train_00000000,VizWiz_train_00000000.jpg,"[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, IT...","[[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, I..."


In [34]:
# coco train anno
imdb_train[:2]

array([{'metadata': 'coco'},
       {'image_name': 'COCO_train2014_000000057870', 'image_id': 40504, 'feature_path': 'COCO_train2014_000000057870.npy', 'caption_id': 787980, 'caption_str': 'A restaurant has modern wooden tables and chairs.', 'caption_tokens': ['<s>', 'a', 'restaurant', 'has', 'modern', 'wooden', 'tables', 'and', 'chairs', '</s>'], 'reference_tokens': [['<s>', 'a', 'restaurant', 'has', 'modern', 'wooden', 'tables', 'and', 'chairs', '</s>'], ['<s>', 'a', 'long', 'restaurant', 'table', 'with', 'rattan', 'rounded', 'back', 'chairs', '</s>'], ['<s>', 'a', 'long', 'table', 'with', 'a', 'plant', 'on', 'top', 'of', 'it', 'surrounded', 'with', 'wooden', 'chairs', '</s>'], ['<s>', 'a', 'long', 'table', 'with', 'a', 'flower', 'arrangement', 'in', 'the', 'middle', 'for', 'meetings', '</s>'], ['<s>', 'a', 'table', 'is', 'adorned', 'with', 'wooden', 'chairs', 'with', 'blue', 'accents', '</s>']]}],
      dtype=object)

In [127]:
imdb_val[:2]

array([{'metadata': 'coco'},
       {'image_name': 'COCO_val2014_000000184613', 'image_id': 2, 'feature_path': 'COCO_val2014_000000184613.npy', 'caption_id': 474921, 'caption_str': 'A child holding a flowered umbrella and petting a yak.', 'caption_tokens': ['<s>', 'a', 'child', 'holding', 'a', 'flowered', 'umbrella', 'and', 'petting', 'a', 'yak', '</s>'], 'reference_tokens': [['<s>', 'a', 'child', 'holding', 'a', 'flowered', 'umbrella', 'and', 'petting', 'a', 'yak', '</s>'], ['<s>', 'a', 'young', 'man', 'holding', 'an', 'umbrella', 'next', 'to', 'a', 'herd', 'of', 'cattle', '</s>'], ['<s>', 'a', 'young', 'boy', 'barefoot', 'holding', 'an', 'umbrella', 'touching', 'the', 'horn', 'of', 'a', 'cow', '</s>'], ['<s>', 'a', 'young', 'boy', 'with', 'an', 'umbrella', 'who', 'is', 'touching', 'the', 'horn', 'of', 'a', 'cow', '</s>'], ['<s>', 'a', 'boy', 'holding', 'an', 'umbrella', 'while', 'standing', 'next', 'to', 'livestock', '</s>']]}],
      dtype=object)

In [133]:
imdb_train_df = pd.DataFrame(imdb_train.tolist())
imdb_train_df["image_id"].min(), imdb_train_df["image_id"].max(), imdb_train_df["caption_id"].min(), imdb_train_df["caption_id"].max()

(40504.0, 123286.0, 26.0, 829719.0)

In [130]:
imdb_val_df = pd.DataFrame(imdb_val.tolist())

In [132]:
imdb_val_df["image_id"].min(), imdb_val_df["image_id"].max(), imdb_val_df["caption_id"].min(), imdb_val_df["caption_id"].max() 

(2.0, 40482.0, 91.0, 829420.0)

In [ ]:
anno_df_arr = np.asarray([*anno_df["result"]])

In [ ]:
# train에 대한 것
#예상: image_id(123287~), caption_id(9278~), feature_path(jpg->npy) 일걸로 예상하지만, 실제 max값이 어떤지 확인해보기
#실제:          123287~            829720~   

# val에 대한 것
#예상: image_id(40483~), caption_id(829421~), feature_path(jpg->npy)

In [88]:
anno_df["image_id"].min(), anno_df["caption_id"].min(), anno_df["image_id"].max(), anno_df["caption_id"].max()

(0, 0, 23430, 117154)

In [87]:
val_anno_df["image_id"].min(), val_anno_df["caption_id"].min(), val_anno_df["image_id"].max(), val_anno_df["caption_id"].max()

(23431, 117155, 31180, 155904)

In [134]:
# index 조정하기 (image_id, caption_id)
global offset_image_idx
offset_image_idx = 123287
def change_image_idx(image_id:int):
    global offset_image_idx    
    return offset_image_idx + image_id

# index 조정하기 (image_id, caption_id)
global offset_caption_idx
offset_caption_idx = 829720
def change_caption_idx(caption_id:int):
    global offset_caption_idx    
    return offset_caption_idx + caption_id

anno_df["image_new_id"] = anno_df["image_id"].map(change_image_idx)
anno_df["caption_new_id"] = anno_df["caption_id"].map(change_caption_idx)

In [135]:
offset_image_idx = 40483
offset_caption_idx = 829421

val_anno_df["image_new_id"] = val_anno_df["image_id"].map(change_image_idx)
val_anno_df["caption_new_id"] = val_anno_df["caption_id"].map(change_caption_idx)

In [136]:
anno_df[["image_new_id", "image_id", "caption_new_id", "caption_id"]][:6]

,image_new_id,image_id,caption_new_id,caption_id
0,123287,0,829720,0
1,123287,0,829721,1
2,123287,0,829722,2
3,123287,0,829723,3
4,123287,0,829724,4
5,123288,1,829725,5


In [137]:
val_anno_df[["image_new_id", "image_id", "caption_new_id", "caption_id"]][:6]

,image_new_id,image_id,caption_new_id,caption_id
0,63914,23431,946576,117155
1,63914,23431,946577,117156
2,63914,23431,946578,117157
3,63914,23431,946579,117158
4,63914,23431,946580,117159
5,63915,23432,946581,117160


In [138]:
def jpg2npy(feature_path:str):
    return feature_path.replace("jpg", "npy")
    
anno_df["feature_path"] = anno_df["file_name"].map(jpg2npy)
val_anno_df["feature_path"] = val_anno_df["file_name"].map(jpg2npy)

In [195]:
anno_df[:2]

,image_id,caption_id,caption_str,image_name,file_name,caption,reference_tokens,result,image_new_id,caption_new_id,feature_path
0,0,0,ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS T...,VizWiz_train_00000000,VizWiz_train_00000000.jpg,"[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, IT...","[[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, I...","{'image_name': 'VizWiz_train_00000000', 'image...",123287,829720,VizWiz_train_00000000.npy
1,0,1,A green and white plastic condiment bottle con...,VizWiz_train_00000000,VizWiz_train_00000000.jpg,"[<s>, A, green, and, white, plastic, condiment...","[[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, I...","{'image_name': 'VizWiz_train_00000000', 'image...",123287,829721,VizWiz_train_00000000.npy


In [196]:
val_anno_df[:2]

,image_id,image_name,file_name,caption_id,caption_str,caption,reference_tokens,result,image_new_id,caption_new_id,feature_path
0,23431,VizWiz_val_00000000,VizWiz_val_00000000.jpg,117155,A computer screen shows a repair prompt on the...,"[<s>, A, computer, screen, shows, a, repair, p...","[[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, I...","{'image_name': 'VizWiz_val_00000000', 'image_i...",63914,946576,VizWiz_val_00000000.npy
1,23431,VizWiz_val_00000000,VizWiz_val_00000000.jpg,117156,a computer screen with a repair automatically ...,"[<s>, a, computer, screen, with, a, repair, au...","[[<s>, ITS, IS, A, BASIL, LEAVES, CONTAINER, I...","{'image_name': 'VizWiz_val_00000000', 'image_i...",63914,946577,VizWiz_val_00000000.npy


In [142]:
# 남겨야 되는 퀄럼들
imdb_train[1].keys()

dict_keys(['image_name', 'image_id', 'feature_path', 'caption_id', 'caption_str', 'caption_tokens', 'reference_tokens'])

In [197]:
new_anno_df = anno_df[['image_name', 'image_new_id', 'feature_path', 'caption_new_id', 'caption_str', 'caption', 'reference_tokens']]
new_val_anno_df = val_anno_df[['image_name', 'image_new_id', 'feature_path', 'caption_new_id', 'caption_str', 'caption', 'reference_tokens']]

In [198]:
new_anno_df.keys()

Index(['image_name', 'image_new_id', 'feature_path', 'caption_new_id',
       'caption_str', 'caption', 'reference_tokens'],
      dtype='object')

In [199]:
new_anno_df.rename(columns={"image_new_id":"image_id", "caption_new_id":"caption_id", "caption":"caption_tokens"}, inplace=True)
new_val_anno_df.rename(columns={"image_new_id":"image_id", "caption_new_id":"caption_id", "caption":"caption_tokens"}, inplace=True)

/tf/notebooks/miniconda3/envs/vqa/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [200]:
new_anno_df.keys()

Index(['image_name', 'image_id', 'feature_path', 'caption_id', 'caption_str',
       'caption_tokens', 'reference_tokens'],
      dtype='object')

In [215]:
# train 데이터 합치기
imdb_vizwiz_train = np.concatenate([imdb_train,new_anno_df.to_dict('records')])
imdb_vizwiz_val = np.concatenate([imdb_val, new_val_anno_df[:-100].to_dict('records')])
imdb_vizwiz_test = np.concatenate([imdb_test, new_val_anno_df[-100:].to_dict('records')])

In [217]:
imdb_train.shape, imdb_val.shape, imdb_test.shape

((414114,), (5001,), (5001,))

In [216]:
imdb_vizwiz_train.shape, imdb_vizwiz_val.shape, imdb_vizwiz_test.shape

((531269,), (43651,), (5101,))

In [219]:
# glob("./data/imdb/coco_captions/imdb_karpathy_train.npy")
np.save("./data/imdb/coco_captions/imdb_vizwiz_train.npy", imdb_vizwiz_train, allow_pickle=True)
np.save("./data/imdb/coco_captions/imdb_vizwiz_test.npy", imdb_vizwiz_test, allow_pickle=True)
np.save("./data/imdb/coco_captions/imdb_vizwiz_val.npy", imdb_vizwiz_val, allow_pickle=True)

In [223]:
!ls -al ./data/imdb/coco_captions/

total 324404
drwxr-xr-x 2 root root       208 Apr  8 10:15 .
drwxr-xr-x 5 root root       124 Apr  8 03:17 ..
-rw-r--r-- 1 root root   4481937 Mar 19 10:47 imdb_karpathy_test.npy
-rw-r--r-- 1 root root 124458897 Mar 19 10:47 imdb_karpathy_train.npy
-rw-r--r-- 1 root root   4487529 Mar 19 10:47 imdb_karpathy_val.npy
-rw-r--r-- 1 root root   4536482 Apr  8 10:15 imdb_vizwiz_test.npy
-rw-r--r-- 1 root root 168654863 Apr  8 10:15 imdb_vizwiz_train.npy
-rw-r--r-- 1 root root  25555472 Apr  8 10:15 imdb_vizwiz_val.npy


In [205]:
imdb_vizwiz_train[414113:414115]

array([{'image_name': 'COCO_train2014_000000475546', 'image_id': 123286, 'feature_path': 'COCO_train2014_000000475546.npy', 'caption_id': 9277, 'caption_str': 'Three women and a man are sitting at a bar', 'caption_tokens': ['<s>', 'three', 'women', 'and', 'a', 'man', 'are', 'sitting', 'at', 'a', 'bar', '</s>'], 'reference_tokens': [['<s>', 'the', 'patrons', 'enjoy', 'their', 'beverages', 'at', 'the', 'bar', '</s>'], ['<s>', 'people', 'having', 'a', 'drink', 'in', 'a', 'basement', 'bar', '</s>'], ['<s>', 'a', 'group', 'of', 'friends', 'enjoys', 'a', 'drink', 'while', 'sitting', 'at', 'a', 'bar', '</s>'], ['<s>', 'group', 'of', 'people', 'drinking', 'wine', 'at', 'a', 'public', 'location', '</s>'], ['<s>', 'three', 'women', 'and', 'a', 'man', 'are', 'sitting', 'at', 'a', 'bar', '</s>']]},
       {'image_name': 'VizWiz_train_00000000', 'image_id': 123287, 'feature_path': 'VizWiz_train_00000000.npy', 'caption_id': 829720, 'caption_str': 'ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS THE NET

In [206]:
a = {'image_name': 'COCO_train2014_000000475546', 'image_id': 123286, 'feature_path': 'COCO_train2014_000000475546.npy', 'caption_id': 9277, 'caption_str': 'Three women and a man are sitting at a bar', 'caption_tokens': ['<s>', 'three', 'women', 'and', 'a', 'man', 'are', 'sitting', 'at', 'a', 'bar', '</s>'], 'reference_tokens': [['<s>', 'the', 'patrons', 'enjoy', 'their', 'beverages', 'at', 'the', 'bar', '</s>'], ['<s>', 'people', 'having', 'a', 'drink', 'in', 'a', 'basement', 'bar', '</s>'], ['<s>', 'a', 'group', 'of', 'friends', 'enjoys', 'a', 'drink', 'while', 'sitting', 'at', 'a', 'bar', '</s>'], ['<s>', 'group', 'of', 'people', 'drinking', 'wine', 'at', 'a', 'public', 'location', '</s>'], ['<s>', 'three', 'women', 'and', 'a', 'man', 'are', 'sitting', 'at', 'a', 'bar', '</s>']]}
a.keys()

dict_keys(['image_name', 'image_id', 'feature_path', 'caption_id', 'caption_str', 'caption_tokens', 'reference_tokens'])

In [207]:
b = {'image_name': 'VizWiz_train_00000000', 'image_id': 123287, 'feature_path': 'VizWiz_train_00000000.npy', 'caption_id': 829720, 'caption_str': 'ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS THE NET WEIGHT TOO.', 'caption_tokens': ['<s>', 'ITS', 'IS', 'A', 'BASIL', 'LEAVES', 'CONTAINER', 'ITS', 'CONTAINS', 'THE', 'NET', 'WEIGHT', 'TOO.', '</s>'], 'reference_tokens': [['<s>', 'ITS', 'IS', 'A', 'BASIL', 'LEAVES', 'CONTAINER', 'ITS', 'CONTAINS', 'THE', 'NET', 'WEIGHT', 'TOO.', '</s>'], ['<s>', 'A', 'green', 'and', 'white', 'plastic', 'condiment', 'bottle', 'containing', 'Basil', 'leaves.', '</s>'], ['<s>', 'Quality', 'issues', 'are', 'too', 'severe', 'to', 'recognize', 'visual', 'content.', '</s>'], ['<s>', 'A', 'bottle', 'of', 'spices', 'in', 'a', 'plastic', 'container', 'laying', 'on', 'a', 'surface.', '</s>'], ['<s>', 'some', 'basil', 'leaves', 'in', 'a', 'container', 'on', 'a', 'counter', '</s>']]}
b.keys()

dict_keys(['image_name', 'image_id', 'feature_path', 'caption_id', 'caption_str', 'caption_tokens', 'reference_tokens'])

In [211]:
imdb_vizwiz_val[5000:5002]

array([{'image_name': 'COCO_val2014_000000296759', 'image_id': 40482, 'feature_path': 'COCO_val2014_000000296759.npy', 'caption_id': 180764, 'caption_str': 'A brown teddy bear wearing a Hawaiian shirt next to another teddy bear.', 'caption_tokens': ['<s>', 'a', 'brown', 'teddy', 'bear', 'wearing', 'a', 'hawaiian', 'shirt', 'next', 'to', 'another', 'teddy', 'bear', '</s>'], 'reference_tokens': [['<s>', 'a', 'brown', 'teddy', 'bear', 'wearing', 'a', 'hawaiian', 'shirt', 'next', 'to', 'another', 'teddy', 'bear', '</s>'], ['<s>', 'two', 'stuffed', 'bears', 'next', 'to', 'each', 'other', 'one', 'with', 'a', 'hawaiian', 'shirt', '</s>'], ['<s>', 'two', 'teddy', 'bears', 'pose', 'for', 'a', 'picture', 'together', '</s>'], ['<s>', 'a', 'store', 'features', 'many', 'types', 'of', 'teddy', 'bears', 'with', 'various', 'outfits', '</s>'], ['<s>', 'a', 'large', 'and', 'a', 'small', 'teddy', 'bear', 'at', 'the', 'teddy', 'bear', 'museum', '</s>']]},
       {'image_name': 'VizWiz_val_00000000', 'imag

In [212]:
a = {'image_name': 'COCO_val2014_000000296759', 'image_id': 40482, 'feature_path': 'COCO_val2014_000000296759.npy', 'caption_id': 180764, 'caption_str': 'A brown teddy bear wearing a Hawaiian shirt next to another teddy bear.', 'caption_tokens': ['<s>', 'a', 'brown', 'teddy', 'bear', 'wearing', 'a', 'hawaiian', 'shirt', 'next', 'to', 'another', 'teddy', 'bear', '</s>'], 'reference_tokens': [['<s>', 'a', 'brown', 'teddy', 'bear', 'wearing', 'a', 'hawaiian', 'shirt', 'next', 'to', 'another', 'teddy', 'bear', '</s>'], ['<s>', 'two', 'stuffed', 'bears', 'next', 'to', 'each', 'other', 'one', 'with', 'a', 'hawaiian', 'shirt', '</s>'], ['<s>', 'two', 'teddy', 'bears', 'pose', 'for', 'a', 'picture', 'together', '</s>'], ['<s>', 'a', 'store', 'features', 'many', 'types', 'of', 'teddy', 'bears', 'with', 'various', 'outfits', '</s>'], ['<s>', 'a', 'large', 'and', 'a', 'small', 'teddy', 'bear', 'at', 'the', 'teddy', 'bear', 'museum', '</s>']]}
a.keys()

dict_keys(['image_name', 'image_id', 'feature_path', 'caption_id', 'caption_str', 'caption_tokens', 'reference_tokens'])

In [213]:
b = {'image_name': 'VizWiz_val_00000000', 'image_id': 63914, 'feature_path': 'VizWiz_val_00000000.npy', 'caption_id': 946576, 'caption_str': 'A computer screen shows a repair prompt on the screen.', 'caption_tokens': ['<s>', 'A', 'computer', 'screen', 'shows', 'a', 'repair', 'prompt', 'on', 'the', 'screen.', '</s>'], 'reference_tokens': [['<s>', 'ITS', 'IS', 'A', 'BASIL', 'LEAVES', 'CONTAINER', 'ITS', 'CONTAINS', 'THE', 'NET', 'WEIGHT', 'TOO.', '</s>'], ['<s>', 'A', 'green', 'and', 'white', 'plastic', 'condiment', 'bottle', 'containing', 'Basil', 'leaves.', '</s>'], ['<s>', 'Quality', 'issues', 'are', 'too', 'severe', 'to', 'recognize', 'visual', 'content.', '</s>'], ['<s>', 'A', 'bottle', 'of', 'spices', 'in', 'a', 'plastic', 'container', 'laying', 'on', 'a', 'surface.', '</s>'], ['<s>', 'some', 'basil', 'leaves', 'in', 'a', 'container', 'on', 'a', 'counter', '</s>']]}
b.keys()

dict_keys(['image_name', 'image_id', 'feature_path', 'caption_id', 'caption_str', 'caption_tokens', 'reference_tokens'])